In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def f_hook(m, i, o):
    print(m)
    print(type(m))
    print("forward_input")
    print(type(i))
    for a in i:
        print(a)
    print("forward_output")
    print(type(o))
    print(o)
    print()

def b_hook(m, i, o):
    print(m)
    print(type(m))
    print("grad_input")
    print(type(i))
    for a in i:
        print(a)
    print("grad_output")
    print(type(o))
    for a in o:
        print(a)
    print()

def tensor_hook(x):
    print(type(x))
    print(x)
    print()

## Linear

In [8]:
x = torch.randn(1, 5)
linear_layer = nn.Linear(5, 3)
relu = nn.ReLU()
linear_layer2 = nn.Linear(3, 2)

linear_layer.register_forward_hook(f_hook)
relu.register_forward_hook(f_hook)
linear_layer2.register_forward_hook(f_hook)
linear_layer.register_backward_hook(b_hook)
relu.register_backward_hook(b_hook)
linear_layer2.register_backward_hook(b_hook)
o1 = linear_layer(x)
o2 = relu(o1)
o3 = linear_layer2(o2)
o4 = 9 - o3.mean()

Linear(in_features=5, out_features=3, bias=True)
<class 'torch.nn.modules.linear.Linear'>
forward_input
<class 'tuple'>
tensor([[ 1.0458, -0.2298,  0.3554,  1.4613,  0.8634]])
forward_output
<class 'torch.Tensor'>
tensor([[-0.3579,  0.4016,  0.7146]], grad_fn=<AddmmBackward>)

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
forward_input
<class 'tuple'>
tensor([[-0.3579,  0.4016,  0.7146]], grad_fn=<AddmmBackward>)
forward_output
<class 'torch.Tensor'>
tensor([[0.0000, 0.4016, 0.7146]], grad_fn=<ReluBackward0>)

Linear(in_features=3, out_features=2, bias=True)
<class 'torch.nn.modules.linear.Linear'>
forward_input
<class 'tuple'>
tensor([[0.0000, 0.4016, 0.7146]], grad_fn=<ReluBackward0>)
forward_output
<class 'torch.Tensor'>
tensor([[ 0.1455, -0.0011]], grad_fn=<AddmmBackward>)



In [9]:
o4.backward()

Linear(in_features=3, out_features=2, bias=True)
<class 'torch.nn.modules.linear.Linear'>
grad_input
<class 'tuple'>
tensor([-0.5000, -0.5000])
tensor([[-0.0478, -0.1044, -0.2227]])
tensor([[-0.0000, -0.0000],
        [-0.2008, -0.2008],
        [-0.3573, -0.3573]])
grad_output
<class 'tuple'>
tensor([[-0.5000, -0.5000]])

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
grad_input
<class 'tuple'>
tensor([[ 0.0000, -0.1044, -0.2227]])
grad_output
<class 'tuple'>
tensor([[-0.0478, -0.1044, -0.2227]])

Linear(in_features=5, out_features=3, bias=True)
<class 'torch.nn.modules.linear.Linear'>
grad_input
<class 'tuple'>
tensor([ 0.0000, -0.1044, -0.2227])
None
tensor([[ 0.0000, -0.1092, -0.2329],
        [-0.0000,  0.0240,  0.0512],
        [ 0.0000, -0.0371, -0.0792],
        [ 0.0000, -0.1526, -0.3254],
        [ 0.0000, -0.0902, -0.1923]])
grad_output
<class 'tuple'>
tensor([[ 0.0000, -0.1044, -0.2227]])



## Conv2d

In [2]:
x = torch.randn(1, 1, 5, 5)
conv_layer = nn.Conv2d(1, 1, 2, 1)
relu = nn.ReLU()
conv_layer2 = nn.Conv2d(1, 1, 2, 1)

conv_layer.register_forward_hook(f_hook)
relu.register_forward_hook(f_hook)
conv_layer2.register_forward_hook(f_hook)
conv_layer.register_backward_hook(b_hook)
relu.register_backward_hook(b_hook)
conv_layer2.register_backward_hook(b_hook)
o1 = conv_layer(x)
o2 = relu(o1)
o3 = conv_layer2(o2)
o4 = 1 - o3.mean()

Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1))
<class 'torch.nn.modules.conv.Conv2d'>
forward_input
<class 'tuple'>
tensor([[[[-1.5590, -2.8136, -0.5423, -1.0532,  0.1237],
          [-1.4775, -0.1077, -0.6187,  0.8025,  1.1835],
          [ 1.4065,  0.4982, -0.4970, -1.4976,  0.5764],
          [ 1.6011,  1.0219, -0.0436, -2.6786, -0.4548],
          [ 0.5924, -0.4832,  0.3975,  0.3580, -0.1329]]]])
forward_output
<class 'torch.Tensor'>
tensor([[[ 1.3766,  0.3055,  0.6854,  0.1169],
         [ 0.0208,  0.1201, -0.3871, -0.1131],
         [-0.2982,  0.0356,  0.1988,  0.2473],
         [-0.5883,  0.1445,  1.0332,  0.2721]]], grad_fn=<SelectBackward>)

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
forward_input
<class 'tuple'>
tensor([[[[ 1.3766,  0.3055,  0.6854,  0.1169],
          [ 0.0208,  0.1201, -0.3871, -0.1131],
          [-0.2982,  0.0356,  0.1988,  0.2473],
          [-0.5883,  0.1445,  1.0332,  0.2721]]]],
       grad_fn=<ThnnConv2DBackward>)
forward_output
<class 'torch

In [3]:
o4.backward()

Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1))
<class 'torch.nn.modules.conv.Conv2d'>
grad_input
<class 'tuple'>
tensor([[[[-0.0017,  0.0393,  0.0393,  0.0409],
          [ 0.0482,  0.0901,  0.0901,  0.0419],
          [ 0.0482,  0.0901,  0.0901,  0.0419],
          [ 0.0498,  0.0508,  0.0508,  0.0010]]]])
tensor([[[[-0.3048, -0.1899],
          [-0.1726, -0.2280]]]])
tensor([-1.])
grad_output
<class 'tuple'>
tensor([[[[-0.1111, -0.1111, -0.1111],
          [-0.1111, -0.1111, -0.1111],
          [-0.1111, -0.1111, -0.1111]]]])

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
grad_input
<class 'tuple'>
tensor([[[[-0.0017,  0.0393,  0.0393,  0.0409],
          [ 0.0482,  0.0901,  0.0000,  0.0000],
          [ 0.0000,  0.0901,  0.0901,  0.0419],
          [ 0.0000,  0.0508,  0.0508,  0.0010]]]])
grad_output
<class 'tuple'>
tensor([[[[-0.0017,  0.0393,  0.0393,  0.0409],
          [ 0.0482,  0.0901,  0.0901,  0.0419],
          [ 0.0482,  0.0901,  0.0901,  0.0419],
          [ 0.0498,  

## RNN

In [4]:
torch.manual_seed(5)
x = torch.randn(1, 2, 5).requires_grad_(True)
rnn_layer = nn.RNN(5, 3, 1, batch_first=True)
softmax = nn.Softmax(dim=1)
 
rnn_layer.register_forward_hook(f_hook)
# softmax.register_forward_hook(f_hook)
rnn_layer.register_backward_hook(b_hook)
# softmax.register_backward_hook(b_hook)
rnn_layer.weight_hh_l0.register_hook(tensor_hook)
rnn_layer.weight_ih_l0.register_hook(tensor_hook)
o1, h1 = rnn_layer(x)
o2 = softmax(h1.squeeze(1))
grad = torch.FloatTensor([[1, 0, 0]])

RNN(5, 3, batch_first=True)
<class 'torch.nn.modules.rnn.RNN'>
forward_input
<class 'tuple'>
tensor([[[-0.4868, -0.6038, -0.5581,  0.6675, -0.1974],
         [ 1.9428, -1.4017, -0.7626,  0.6312, -0.8991]]], requires_grad=True)
forward_output
<class 'tuple'>
tensor([[[ 0.7737,  0.6712,  0.7200],
         [ 0.9293,  0.0850, -0.2211]]], grad_fn=<TransposeBackward1>)
tensor([[[ 0.9293,  0.0850, -0.2211]]], grad_fn=<StackBackward>)



In [5]:
l = F.cross_entropy(o1.view(-1, 3), torch.LongTensor([1, 0]))
l.backward()

RNN(5, 3, batch_first=True)
<class 'torch.nn.modules.rnn.RNN'>
grad_input
<class 'tuple'>
tensor([[[ 0.1754, -0.3417,  0.1662]],

        [[-0.2137,  0.1231,  0.0906]]])
grad_output
<class 'tuple'>
tensor([[[ 0.1754, -0.3417,  0.1662],
         [-0.2137,  0.1231,  0.0906]]])

<class 'torch.Tensor'>
tensor([[-0.0225, -0.0196, -0.0210],
        [ 0.0945,  0.0820,  0.0880],
        [ 0.0667,  0.0579,  0.0621]])
<class 'torch.Tensor'>
tensor([[-0.0849,  0.0058, -0.0101,  0.0203,  0.0148],
        [ 0.3266, -0.0606,  0.0091, -0.0452, -0.0737],
        [ 0.1169, -0.1835, -0.1237,  0.1238, -0.0980]])
